In [1]:
!nvidia-smi

Tue Feb 15 06:22:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   44C    P0    27W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install timm==0.4.5
import IPython.display as ipd
!pip install scikit-image



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
!conda install -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension


Solving environment: done

# All requested packages already installed.

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../lib'))
import config
import config_DK_AST
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
from torchvision import datasets
import librosa
import librosa.display
import os
import skimage.util
import pickle
import math
import collections
import pandas as pd
from sklearn.utils import shuffle
from scipy import signal
import scipy
import matplotlib.pyplot as plt
from scipy.signal import butter , filtfilt
import scipy.io as sio
import scipy.io.wavfile
import copy
from PyTorch import config_pytorch
from PyTorch.runTorch_AST_DK import ASTModel
from evaluate import get_results


In [5]:
# Defining global variables to standardize.

train_data_mean = 0
train_data_sd = 0
# set this to True if you want to call the bandpass filter
call_filter = True

print("The mean of the training data = " + str(train_data_mean))
print("The SD of the training data = " + str(train_data_sd))


The mean of the training data = 0
The SD of the training data = 0


In [6]:
!nvidia-smi

Tue Feb 15 06:23:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   43C    P0    27W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Data Related Functions - Reshaping and Feature Extraction from .wav

In [7]:
def get_feat(data_df, data_dir, rate, min_duration, n_feat , filter_signal = call_filter):
    ''' Returns features extracted with Librosa. A list of features, with the number of items equal to the number of input recordings'''
    #dk added
    #print("inside get_feat")
    X = []
    y = []
    bugs = []
    idx = 0
    skipped_files = []
    
    print("data_dir = " + str(data_dir))
    print("rate = " + str(rate))
    print("min_duration = " + str(min_duration))
    print("n_feat = " + str(n_feat))
    
    
    for row_idx_series in data_df.iterrows():
        #print("row_idx_series..." + str(row_idx_series))
        idx+=1
        #print("idx = " + str(idx))
        if idx % 100 == 0:
            print('Completed', idx, 'of', len(data_df))
        row = row_idx_series[1]
        #print("row->" + str(row))
        label_duration = row['length']
        #print("label_duration = " +str(label_duration))
        if label_duration > min_duration:
            #print("if label_duration > min_duration...")
            _, file_format = os.path.splitext(row['name'])
            filename = os.path.join(data_dir, str(row['id']) + file_format)
            #print("filename = " + str(filename))
            length = librosa.get_duration(filename = filename)
            #print("length from librosa = " + str(length) )
#             assert math.isclose(length,label_duration, rel_tol=0.01), "File: %s label duration (%.4f) does not match audio length (%.4f)" % (row['path'], label_duration, length)
            
            if math.isclose(length,label_duration, rel_tol=0.01):
                #print("if math.isclose(length,label_duration....")
                signal_file, rate = librosa.load(filename, sr=rate)
                #print("BEFORE Calling filter....")
                #print("Sorted raw signal is -> " + str(np.sort(signal_file)[::-1] ))
                
                if filter_signal == True :
                    filtered_sig = create_filter(file = filename)
                else:
                    filtered_sig = signal_file
                #print("Sorted Filtered SIgnal is ----" + str(np.sort(filtered_sig)[::-1] ))
                
                feat = librosa.feature.melspectrogram(filtered_sig, sr=rate, n_mels=n_feat) 
                #print("After librosa.feature.melspectrogram . Shape of feat ->" + str(feat.shape))
                feat = librosa.power_to_db(feat, ref=np.max)
                #print("After librosa.power_to_db . Shape of feat ->" + str(feat.shape))
                  
                #librosa.display.specshow(feat)
                if config_DK_AST.norm_per_sample:
                    #print("Attempting to standardize..")
                    #print("mean of the filtered signal - " + str(np.mean(feat)))
                    #print("SD of the filtered signal - " + str(np.std(feat)))
                    #feat = (feat-np.mean(feat))/np.std(feat)                
                    #print("shape of feat...")
                    print("..")
                    #print(feat.shape)
                X.append(feat)
                if row['sound_type'] == 'mosquito':
                    y.append(1)
                elif row['sound_type']:  # Condition to check we are not adding empty (or unexpected) labels as 0
                    y.append(0)
            else:
                print("File: %s label duration (%.4f) does not match audio length (%.4f)" % (row['name'], label_duration, length))
                bugs.append([row['name'], label_duration, length])
                
        else:
            #print("inside skipped files")
            skipped_files.append([row['id'], row['name'], label_duration])
           # print("skipped file = " + str(skipped_files))
            
    #print("length X = " + str(len(X)))
    #print("length y = " + str(len(y)))
    #print("before returning from get_feat skipped file = " + str(skipped_files)  )
    return X, y, skipped_files, bugs


In [8]:
def reshape_feat(feats, labels, win_size, step_size):
    '''Reshaping features from get_feat to be compatible for classifiers expecting a 2D slice as input. Parameter `win_size` is 
    given in number of feature windows (in librosa this is the hop length divided by the sample rate.)
    Can code to be a function of time and hop length instead in future.'''
    from statistics import mean
    #print("inside reshape_feat....")
    #print("Length Input  = " + str(len(feats)))
    feats_windowed_array = []
    labels_windowed_array = []
    for idx, feat in enumerate(feats):
        #print("shape of feat = " + str(np.shape(feat)))
        if np.shape(feat)[1] < win_size:
            print('Length of recording shorter than supplied window size.') 
            pass
        else:
            #create a window
            feats_windowed = skimage.util.view_as_windows(feat.T, (win_size,np.shape(feat)[0]), step=step_size)
            labels_windowed = np.full(len(feats_windowed), labels[idx])
            feats_windowed_array.append(feats_windowed)
            labels_windowed_array.append(labels_windowed)
          
    #numpy.vstack() function is used to stack the sequence of input arrays vertically to make a single array.
    #print("Length of Feat windowed arraay = " + str(len(feats_windowed_array)))
    #print("Mean of features = " + str(mean(feats_windowed_array)))
    #print("Std Deviation of features = ")
    return np.vstack(feats_windowed_array), np.hstack(labels_windowed_array)

In [9]:
def get_train_test_from_df(df_train, df_test_A, df_test_B, debug=False):
    #dk added
    #print("Inside get_train_test_from_df ")
    #print("Creating Pickle file.. ")
    
    
    pickle_name_train = 'log_mel_feat_train_AST'+str(config.n_feat)+'_win_'+str(config.win_size)+'_step_'+str(config.step_size)+'_norm_'+str(config.norm_per_sample)+'.pickle'
    print("pickle_name_train = " + str(pickle_name_train))
    # step = window for test (no augmentation of test):
    pickle_name_test = 'log_mel_feat_test_AST'+str(config.n_feat)+'_win_'+str(config.win_size)+'_step_'+str(config.win_size)+'_norm_'+str(config.norm_per_sample)+'.pickle'
    print("pickle_name_test = " + str(pickle_name_test))
    if not os.path.isfile(os.path.join(config.dir_out_MED, pickle_name_train)):
        #dk added
        print("Inside if...when pickle file is not found")
        print('Extracting training features...')
        X_train, y_train, skipped_files_train, bugs_train = get_feat(data_df=df_train, data_dir = config_DK_AST.data_dir,
                                                                     rate=config_DK_AST.rate, min_duration=config_DK_AST.min_duration,
                                                                     n_feat=config.n_feat,filter_signal = call_filter)
        #print("After returning from get_feat..")
        #print("skipped files = " + str(skipped_files_train))
        #print("X_Train = " + str(X_train))
        print("X_Train len  = " +str(len(X_train)))
        
        print("before calling reshape....")
        X_train, y_train = reshape_feat(X_train, y_train, config_DK_AST.win_size, config_DK_AST.step_size)
        #print("After X_train re shape = " + str(X_train.shape))
        
        #print("THIS IS WHERE STANDARDIZATION CAN GO IN....")

        log_mel_feat_train = {'X_train':X_train, 'y_train':y_train, 'bugs_train':bugs_train}
        

        if debug:
            print('Bugs train', bugs_train)
        
        with open(os.path.join(config.dir_out_MED, pickle_name_train), 'wb') as f:
            pickle.dump(log_mel_feat_train, f, protocol=4)
            print('Saved features to:', os.path.join(config.dir_out_MED, pickle_name_train))

    else:
        print('Loading training features found at:', os.path.join(config.dir_out_MED, pickle_name_train))
        with open(os.path.join(config.dir_out_MED, pickle_name_train), 'rb') as input_file:
            log_mel_feat = pickle.load(input_file)
            X_train = log_mel_feat['X_train']
            #print("X_train shape = " + str(X_train.shape))
          
            y_train = log_mel_feat['y_train']

    if not os.path.isfile(os.path.join(config.dir_out_MED, pickle_name_test)):
        print('Extracting test features...')

        X_test_A, y_test_A, skipped_files_test_A, bugs_test_A = get_feat(data_df= df_test_A, data_dir = config_DK_AST.data_dir,
                                                                         rate=config_DK_AST.rate, min_duration=config_DK_AST.min_duration,
                                                                         n_feat=config_DK_AST.n_feat,filter_signal = call_filter)
        
        X_test_B, y_test_B, skipped_files_test_B, bugs_test_B = get_feat(data_df= df_test_B, data_dir = config_DK_AST.data_dir,
                                                                         rate=config_DK_AST.rate, min_duration=config_DK_AST.min_duration,
                                                                         n_feat=config_DK_AST.n_feat,filter_signal = call_filter)
        X_test_A, y_test_A = reshape_feat(X_test_A, y_test_A, config_DK_AST.win_size, config_DK_AST.win_size)  # Test should be strided with step = window.
        X_test_B, y_test_B = reshape_feat(X_test_B, y_test_B, config_DK_AST.win_size, config_DK_AST.win_size)  
        
        log_mel_feat_test = {'X_test_A':X_test_A, 'X_test_B':X_test_B, 'y_test_A':y_test_A, 'y_test_B':y_test_B}
        
        #print("Shape of X_TEST A = " + str(X_test_A.shape))
        #print("Shape of X_TEST B = " + str(X_test_B.shape))

        if debug:
            print('Bugs test A', bugs_test_A)
            print('Bugs test B', bugs_test_B)

        
        with open(os.path.join(config_DK_AST.dir_out_MED, pickle_name_test), 'wb') as f:
            pickle.dump(log_mel_feat_test, f, protocol=4)
            print('Saved features to:', os.path.join(config_DK_AST.dir_out_MED, pickle_name_test))
    else:
        print('Loading test features found at:', os.path.join(config_DK_AST.dir_out_MED, pickle_name_test))
        with open(os.path.join(config_DK_AST.dir_out_MED, pickle_name_test), 'rb') as input_file:
            log_mel_feat = pickle.load(input_file)

            X_test_A = log_mel_feat['X_test_A']
            y_test_A = log_mel_feat['y_test_A']
            X_test_B = log_mel_feat['X_test_B']
            y_test_B = log_mel_feat['y_test_B']


    return X_train, y_train, X_test_A, y_test_A, X_test_B, y_test_B




def get_test_from_df(df_test_A, df_test_B, debug=False, pickle_name=None):
    
    if not pickle_name:
        pickle_name_test = 'log_mel_feat_test_'+str(config.n_feat)+'_win_'+str(config.win_size)+'_step_'+str(config.win_size)+'_norm_'+str(config.norm_per_sample)+'.pickle'
    else:
        pickle_name_test = pickle_name
    
    if not os.path.isfile(os.path.join(config.dir_out_MED, pickle_name_test)):
        print('Extracting test features...')

        X_test_A, y_test_A, skipped_files_test_A, bugs_test_A = get_feat(data_df= df_test_A, data_dir = config_DK_AST.data_dir,
                                                                         rate=config_DK_AST.rate, min_duration=config_DK_AST.min_duration,
                                                                         n_feat=config_DK_AST.n_feat,filter_signal = call_filter)
        X_test_B, y_test_B, skipped_files_test_B, bugs_test_B = get_feat(data_df= df_test_B, data_dir = config_DK_AST.data_dir,
                                                                         rate=config_DK_AST.rate, min_duration=config_DK_AST.min_duration,
                                                                         n_feat=config_DK_AST.n_feat,filter_signal = call_filter)
        X_test_A, y_test_A = reshape_feat(X_test_A, y_test_A, config_DK_AST.win_size, config_DK_AST.win_size)  # Test should be strided with step = window.
        X_test_B, y_test_B = reshape_feat(X_test_B, y_test_B, config_DK_AST.win_size, config_DK_AST.win_size)  
        print("STANDARDIZATION CAN GO HERE FOR A and B")
        
        log_mel_feat_test = {'X_test_A':X_test_A, 'X_test_B':X_test_B, 'y_test_A':y_test_A, 'y_test_B':y_test_B}

        if debug:
            print('Bugs test A', bugs_test_A)
            print('Bugs test B', bugs_test_B)

        
        with open(os.path.join(config_DK_AST.dir_out_MED, pickle_name_test), 'wb') as f:
            pickle.dump(log_mel_feat_test, f)
            print('Saved features to:', os.path.join(config_DK_AST.dir_out_MED, pickle_name_test))
    else:
        print('Loading test features found at:', os.path.join(config_DK_AST.dir_out_MED, pickle_name_test))
        with open(os.path.join(config_DK_AST.dir_out_MED, pickle_name_test), 'rb') as input_file:
            log_mel_feat = pickle.load(input_file)

            X_test_A = log_mel_feat['X_test_A']
            y_test_A = log_mel_feat['y_test_A']
            X_test_B = log_mel_feat['X_test_B']
            y_test_B = log_mel_feat['y_test_B']


    return X_test_A, y_test_A, X_test_B, y_test_B



In [10]:
# The function below applies a band pass filter
def create_filter(file , lower_bnd = 300 , upper_bnd = 1000 ,samprate = config_DK_AST.rate , transw = .1 , plot_spec = False, save = False):
    samples, sample_rate = librosa.load(file, sr=None)
    aud_durn = librosa.get_duration(filename = file)
    
    lower_bnd = lower_bnd # Hz
    upper_bnd = upper_bnd # Hz
    transw = .1
    samprate = config_DK_AST.rate
    N = samprate*aud_durn
    ord_param = 30
    # noise is the original sound file
    noise = samples
    timevec = np.arange(0,len(noise))/samprate
    # the better way to filter...
    
    ### first apply a high-pass filter

    forder = int(ord_param*samprate/lower_bnd)+1
    filtkern = signal.firwin(forder,lower_bnd,pass_zero=False,fs=samprate)
    # spectrum of kernel
    hz = np.linspace(0,samprate/2,int(np.floor(len(filtkern)/2)+1))
    filterpow = np.abs(scipy.fftpack.fft(filtkern))**2
    plt.plot(hz,filterpow[:len(hz)],'k')
    
    # zero-phase-shift filter with reflection
    fnoise = signal.filtfilt(filtkern,1,noise)
    ### repeat for low-pass filter
    forder = int(ord_param*samprate/upper_bnd)+1
    filtkern = signal.firwin(forder,upper_bnd,fs=samprate,pass_zero=True)
    
    # spectrum of kernel
    hz = np.linspace(0,samprate/2,int(np.floor(len(filtkern)/2)+1))
    filterpow = np.abs(scipy.fftpack.fft(filtkern))**2
    ## now filter the filtered signal
    fnoise = signal.filtfilt(filtkern,1,fnoise)
    
    # if we want to save the filtered file
    if save == True:
        import soundfile as sf
        new_name = file + "filt" + ".wav"
        sf.write(new_name, fnoise, samplerate = samprate)
      
    
    # the below code will run if the plot argument is true while calling the function
    if plot_spec == True:
        plotcurves(hz = hz ,lower_bnd = lower_bnd, upper_bnd = upper_bnd,timevec = timevec,noise = noise)
        print("Spectrogram of filtered signal is ....")
        plot_specgram(y = fnoise)
        
    return(fnoise)


## Modeling 

In [11]:
def build_dataloader_ast(x_train, y_train, x_val=None, y_val=None, shuffle=True, n_channels=1):
    
    x_train = torch.from_numpy(x_train).float()
    y_train = torch.tensor(y_train).float()
    
    if n_channels == 3:
        x_train = x_train.repeat(1,3,1,1)  # Repeat across 3 channels to match ResNet pre-trained model expectation
    
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=config_pytorch.batch_size, shuffle=shuffle,pin_memory = True)
    
    mean=[]
    std=[]
    
    """""for i, (x_tr, y_tr) in enumerate(train_loader):
        cur_mean = torch.mean(x_tr)
        cur_std = torch.std(x_tr)
        mean.append(cur_mean)
        std.append(cur_std)
        #print(cur_mean, cur_std)"""
    
    
    train_data_mean = np.mean(mean)
    train_data_sd = np.mean(std)
    #print("Inside the dataloader. Modifying the global var for train data mean and std dev.")
    #print("The mean of the training data = " + str(train_data_mean))
    #print("The SD of the training data = " + str(train_data_sd))

    
    if x_val is not None:
        x_val = torch.tensor(x_val).float()
        if n_channels == 3:
            x_val = x_val.repeat(1,3,1,1)
        y_val = torch.tensor(y_val).float()
        val_dataset = TensorDataset(x_val, y_val)
        val_loader = DataLoader(val_dataset, batch_size=config_pytorch.batch_size, shuffle=shuffle,pin_memory = True)

        return train_loader, val_loader
    return train_loader 

In [12]:
def train_model_ast(x_train, y_train, x_val=None, y_val=None, model = ASTModel()):
    
    
    if x_val is not None:  # TODO: check dimensions when supplying validation data.
        train_loader, val_loader = build_dataloader_ast(x_train, y_train, x_val, y_val, n_channels = model.n_channels)
    
    else:
        train_loader = build_dataloader_ast(x_train, y_train, n_channels = model.n_channels)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'Training on {device}')

    if torch.cuda.device_count() > 1:
        print("Using data parallel")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))

    model = model.to(device)
    print("Model Device= " +str(next(model.parameters()).is_cuda))

    # Change compatibility to other loss function, cross-test with main.
    criterion = nn.BCEWithLogitsLoss()
    
    #criterion = nn.CrossEntropyLoss()
    optimiser = optim.Adam(model.parameters(), lr=config_pytorch.lr)
    #print("Optim Device= " +str(optimiser.device))
    

    all_train_loss = []
    all_train_acc = []
    all_val_loss = []
    all_val_acc = []
    best_val_loss = np.inf
    best_val_acc = -np.inf

    # best_train_loss = np.inf
    best_train_acc = -np.inf

    best_epoch = -1
    checkpoint_name = None
    overrun_counter = 0

    for e in range(config_pytorch.epochs):
        train_loss = 0.0
        model.to(device).train()

        all_y = []
        all_y_pred = []
        
        for idx,(x,labels) in enumerate(train_loader):
            # The model needs input in the form of(batchsize, time, freq . Reshaping the features below)           
            bat_size = x.shape[0]
            time_dim = x.shape[2]
            freq_dim = x.shape[3]
            #print("Freq_dim = " +str(freq_dim))
            #here you'll send it to model 
            x_reshaped = x.reshape(bat_size,time_dim,freq_dim).float().cuda()
            # this is recommended in paper.
            #x_reshaped = (x_reshaped - train_data_mean)/(train_data_sd)
            y = labels.reshape(-1,1).cuda()
            #print("Lables = " +str(y))
            #print("Shape of Y = " + str(y.shape))
            #print("Unsqueeze Y = " + str(y))
            #print(x_reshaped.shape)
            
            
            if len(x) == 1:
                x = x[0]

            optimiser.zero_grad()
            x_reshaped.cuda()
            #print("X_REshaped device = " +str(x_reshaped.device))
            y.cuda()
            #print("Y device = " +str(y.device))
            
            y_pred = model(x_reshaped.to(device).detach())
            #print("Y Pred Device = " +str(y_pred.device))
            #print("Y Pred shape = " +str(y_pred.shape))
            #print("Y  shape = " +str(y.shape))
            #print("Y_pred = " + str(y_pred))
            #print("sigmoid of Y_pred = " +str(m(y_pred)))
                       
            loss = criterion(y_pred, y)
            #print("loss  = " +str(loss.device))
            loss.backward()
            optimiser.step()

            #print("Current train loss before adding the loss = " + str(train_loss))
            train_loss += loss.item()
            #print("train loss - >" + str(loss.item()))
            #print("train loss/len(train_loader)-> " + str(train_loss/len(train_loader)))
            all_y.append(y.cpu().detach())
            all_y_pred.append(y_pred.cpu().detach())
            
            del x
            del y

        all_train_loss.append(train_loss/len(train_loader))

        all_y = torch.cat(all_y)
        all_y_pred = torch.cat(all_y_pred)
        train_acc = accuracy_score(all_y.numpy(), (all_y_pred.numpy() > 0.5).astype(float))
        all_train_acc.append(train_acc)


        # Can add more conditions to support loss instead of accuracy. Use *-1 for loss inequality instead of acc
        if x_val is not None:
            val_loss, val_acc = test_model(model, val_loader, criterion, 0.5, device=device)
            all_val_loss.append(val_loss)
            all_val_acc.append(val_acc)

            acc_metric = val_acc
            best_acc_metric = best_val_acc
        else:
            acc_metric = train_acc
            best_acc_metric = best_train_acc
        if acc_metric > best_acc_metric:  
            # if checkpoint_name is not None:
                # os.path.join(os.path.pardir, 'models', 'pytorch', checkpoint_name)

            checkpoint_name = f'model_e{e}_{datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}.pth'

            torch.save(model.state_dict(), os.path.join(config.model_dir, 'pytorch', checkpoint_name))
            print('Saving model to:', os.path.join(config.model_dir, 'pytorch', checkpoint_name)) 
            best_epoch = e
            best_train_acc = train_acc
            best_train_loss = train_loss
            if x_val is not None:
                best_val_acc = val_acc
                best_val_loss = val_loss
            overrun_counter = -1

        overrun_counter += 1
        if x_val is not None:
            print('Epoch: %d, Train Loss: %.8f, Train Acc: %.8f, Val Loss: %.8f, Val Acc: %.8f, overrun_counter %i' % (e, train_loss/len(train_loader), train_acc, val_loss/len(val_loader), val_acc,  overrun_counter))
        else:
            print('Epoch: %d, Train Loss: %.8f, Train Acc: %.8f, overrun_counter %i' % (e, train_loss/len(train_loader), train_acc, overrun_counter))
        if overrun_counter > config_pytorch.max_overrun:
            break
    return model




---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=24


In [13]:
def evaluate_model(model, X_test, y_test, n_samples):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'Evaluating on {device}')
    print("n_samples = " +str(n_samples))

    
    x_test = torch.tensor(X_test).float()
    if model.n_channels == 3:
        x_test = x_test.repeat(1,3,1,1)

    y_test = torch.tensor(y_test).float()
    test_dataset = TensorDataset(x_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
        
    y_preds_all = np.zeros([n_samples, len(y_test), 2])
    model.eval() # Important to not leak info from batch norm layers and cause other issues

    for n in range(n_samples):
        all_y_pred = []
        all_y = []
        
        for idx,(x, labels ) in enumerate(test_loader):
            # The model needs input in the form of(batchsize, time, freq . Reshaping the features below)           
            #print("bat_size = " + str(idx))
            bat_size = x.shape[0]
            time_dim = x.shape[2]
            freq_dim = x.shape[3]
            #print("Freq_dim = " +str(freq_dim))
            #here you'll send it to model 
            x_reshaped = x.reshape(bat_size,time_dim,freq_dim).float()
            #print("Reshaped_X shape = " + str(x_reshaped.shape))
            # this is recommended in paper.
            #x_reshaped = (x_reshaped - train_data_mean)/(train_data_sd)
            y = labels.reshape(-1,1)
            #print("Lables = " +str(y))
            #print("Shape of Y = " + str(y.shape))
            #print("Unsqueeze Y = " + str(y))
            #print(x_reshaped.shape)
            
            x=  x_reshaped.to(device)
            y = y.to(device)

            y_pred = model(x).squeeze()
            #print("Shape of Y_pred = " + str(y_pred.shape))
            all_y.append(y.cpu().detach())

            all_y_pred.append(y_pred.cpu().detach())

            del x
            del y
            del y_pred

        all_y_pred = torch.cat(all_y_pred)
        all_y = torch.cat(all_y)

        y_preds_all[n,:,1] = np.array(all_y_pred)
        y_preds_all[n,:,0] = 1-np.array(all_y_pred) # Check ordering of classes (yes/no)
        test_acc = accuracy_score(all_y.numpy(), (all_y_pred.numpy() > 0.5).astype(float))
        # print(test_acc)
    return y_preds_all



In [14]:
def evaluate_model_aggregated(model, X_test, y_test, n_samples):
    ''' Generate predictions for VGGish features (Feat. A) rescaled to time window of 1.92 second features (Feat. B)'''
    preds_aggregated_by_mean = []
    y_aggregated_prediction_by_mean = []
    y_target_aggregated = []
    
    for idx, recording in enumerate(X_test):
        n_target_windows = len(recording)//2  # Calculate expected length: discard edge
        y_target = np.repeat(y_test[idx],n_target_windows) # Create y array of correct length
        preds = evaluate_model(model, recording, np.repeat(y_test[idx],len(recording)),n_samples) # Sample BNN
#         preds = np.mean(preds, axis=0) # Average across BNN samples
#         print(np.shape(preds))
        preds = preds[:,:n_target_windows*2,:] # Discard edge case
#         print(np.shape(preds))
#         print('reshaping')
        preds = np.mean(preds.reshape(len(preds),-1,2,2), axis=2) # Average every 2 elements, keep samples in first dim
#         print(np.shape(preds))
        preds_y = np.argmax(preds)  # Append argmax prediction (label output)
        y_aggregated_prediction_by_mean.append(preds_y)
        preds_aggregated_by_mean.append(preds)  # Append prob (or log-prob/other space)
        y_target_aggregated.append(y_target)  # Append y_target
#     return preds_aggregated_by_mean, y_aggregated_prediction_by_mean, y_target_aggregated
    return np.hstack(preds_aggregated_by_mean), np.concatenate(y_target_aggregated)


In [15]:
def test_model(model, test_loader, criterion, class_threshold=0.5, device=None):
    with torch.no_grad():
        if device is None:
            torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
        test_loss = 0.0
        model.eval()
        
        all_y = []
        all_y_pred = []
        counter = 1
        for idx,(x,labels) in enumerate(test_loader):
            bat_size = x.shape[0]
            #print("bat_size->" + str(bat_size))
            time_dim = x.shape[2]
            freq_dim = x.shape[3]
            #print("time_dim = " +str(time_dim))
            #print("Freq_dim = " +str(freq_dim))
            #here you'll send it to model 
            x_reshaped = x.reshape(bat_size,time_dim,freq_dim).float().cuda()
            #Doing the same standardization on the test data
            #x_reshaped = (x_reshaped - train_data_mean)/(train_data_sd)
            
            y = labels.reshape(-1,1).cuda()
                      
            
            y_pred = y_pred = model(x_reshaped.to(device).detach())
            
            loss = criterion(y_pred, y)

            test_loss += loss.item()
            
            all_y.append(y.cpu().detach())
            all_y_pred.append(y_pred.cpu().detach())
            
            del x
            del y
            del y_pred
            
            counter +=1

        all_y = torch.cat(all_y)
        all_y_pred = torch.cat(all_y_pred)
        
        test_loss = test_loss/len(test_loader)
        test_acc = accuracy_score(all_y.numpy(), (all_y_pred.numpy() > class_threshold).astype(float))
    
    
    return test_loss, test_acc

## Loading DATA and Training

In [16]:
library = 'PyTorch'

if library == 'PyTorch':
    from PyTorch.runTorch_AST_DK import (ASTModel)

In [17]:
df = pd.read_csv(config_DK_AST.data_df)

# To be kept: please do not edit the test set: these paths select test set A, test set B as described in the paper
idx_test_A = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'field')
idx_test_B = np.logical_and(df['country'] == 'UK', df['location_type'] == 'culture')
idx_train = np.logical_not(np.logical_or(idx_test_A, idx_test_B))
df_test_A = df[idx_test_A]
df_test_B = df[idx_test_B]


df_train = df[idx_train]

# Modify by addition or sub-sampling of df_train here
# df_train ... 

# Assertion to check that train does NOT appear in test:
assert len(np.where(pd.concat([df_train,df_test_A,
                               df_test_B]).duplicated())[0]) == 0, 'Train dataframe contains overlap with Test A, Test B'


In [ ]:
X_train, y_train, X_test_A, y_test_A, X_test_B, y_test_B = get_train_test_from_df(df_train, df_test_A, df_test_B, debug=True)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle = False)

pickle_name_train = log_mel_feat_train_AST128_win_30_step_5_norm_True.pickle
pickle_name_test = log_mel_feat_test_AST128_win_30_step_30_norm_True.pickle
Inside if...when pickle file is not found
Extracting training features...
data_dir = ../data/audio
rate = 16000
min_duration = 0.96
n_feat = 128
Completed 100 of 7934
Completed 200 of 7934
Completed 300 of 7934
Completed 400 of 7934
Completed 500 of 7934
Completed 600 of 7934
Completed 700 of 7934


In [ ]:
!nvidia-smi

In [ ]:
# UNCOMMENT THE LINE BELOW IF YOU WANT TO TRAIN THE MODEL

model = train_model_ast(X_train, y_train, X_val, y_val, model=ASTModel())








## Evaluation

In [ ]:
def load_model(filepath, model=ASTModel()):
    # Instantiate model to inspect
    print("Filepath = " + str(filepath))
    print("model = " +str(model))
    device = torch.device('cuda:0' if torch.cuda.is_available() else torch.device("cpu"))
    print(f'Training on {device}')
        
    if torch.cuda.device_count() > 1:
        print("Using data parallel")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    model = model.to(device)
    # Load trained parameters from checkpoint (may need to download from S3 first)


    if torch.cuda.is_available():
        map_location=lambda storage, loc: storage.cuda()
    else:
        map_location='cpu'
        
    checkpoint = model.load_state_dict(torch.load(filepath))

    return model

In [ ]:
#mlp = MLP()
#mlp.load_state_dict(torch.load(save_path))


model_init=ASTModel()
model_init.eval()
path = '../outputs/models/pytorch/'
model_name = 'model_e2_2022_02_14_21_57_53.pth'


#checkpoint = model.load_state_dict(torch.load(filepath))
trained_model = load_model(path + model_name , model = model_init)


# Dummy data for evaluation

#creating random data to test 
#f1 = np.random.randn(128,1075)
#f2 = np.random.randn(128,242)
#f3 = np.random.randn(128,234)
#f4 = np.ranadom.randn(128,263)

# replicates get_feat
#feats = [f1,f2,f3,f4]
#labels = [0,1,0,1]



## TEST A

In [ ]:
feat_type = 'FeatA'
if feat_type == 'FeatA':
    p, yt = evaluate_model_aggregated(model, X_test_A, y_test_A, 30)  # Aggregate windows from feature list (0.96->1.92 s)
    PE, MI, log_prob = get_results(p, yt, filename = feat_type + '_' + model_name +'_Test_A')
elif feat_type == 'FeatB':
    y_preds_all = evaluate_model(model, X_test_A, y_test_A, 30)  # Predict directly over feature windows (1.92 s)
    PE, MI, log_prob = get_results(y_preds_all, y_test_A, filename = feat_type + '_' + model_name +'_Test_A')

    
    

## TEST B

In [ ]:
feat_type = 'FeatB'
if feat_type == 'FeatA':
    
    p, yt = evaluate_model_aggregated(model, X_test_B, y_test_B, 30)  # Aggregate windows from feature list (0.96->1.92 s)
    PE, MI, log_prob = get_results(p, yt, filename = feat_type + '_' + model_name +'_Test_B')
elif feat_type == 'FeatB':
    y_preds_all = evaluate_model(trained_model, X_test_B, y_test_B, 30)  # Predict directly over feature windows (1.92 s)
    PE, MI, log_prob = get_results(y_preds_all, y_test_B, filename = feat_type + '_' + model_name +'_Test_B')


## TEST SECTION-->use for Test data(Synthetic) creation  

In [ ]:
#creating random data to test 
# This code below replicates get feat
f1 = np.random.randn(128,1075)
f2 = np.random.randn(128,242)
f3 = np.random.randn(128,234)
f4 = np.random.randn(128,263)

feats = [f1,f2,f3,f4]
labels = [0,1,0,1]
type(feats)
sum_all = 0
feats_windowed_array = []
labels_windowed_array = []
for idx, feat in enumerate(feats):
    print("idx = " + str(idx))
    print("shape of feat = " + str(np.shape(feat)))
    print("current sum of array elem = " + str(np.sum(feat)))
    sum_all += np.sum(feat)
    print("Running/Global Sum =  " + str((sum_all)))
    
    
    if np.shape(feat)[1] < 30:
        print('Length of recording shorter than supplied window size.') 
        pass
    else:
        #create a window
        feats_windowed = skimage.util.view_as_windows(feat.T, (30,np.shape(feat)[0]), step=5)
        labels_windowed = np.full(len(feats_windowed), labels[idx])
        feats_windowed_array.append(feats_windowed)
        labels_windowed_array.append(labels_windowed)
          
    #numpy.vstack() function is used to stack the sequence of input arrays vertically to make a single array.
    print("Length of Feat windowed arraay = " + str(len(feats_windowed_array)))
    #print("Mean of features = " + str(mean(feats_windowed_array)))
    print("Std Deviation of features = ")
features =  np.vstack(feats_windowed_array)
lbl = np.hstack(labels_windowed_array)

x_tr , y_tr = features , lbl

print("Shape of x_tr = " + str(x_tr.shape))
print("Shape of y_tr = " + str(y_tr.shape))

num_elem = int(x_tr.shape[0])
print("num_spec = " + str(num_elem))
ldr = build_dataloader_ast(x_tr , y_tr)
mean=[]
std=[]
for i, (x_tr, y_tr) in enumerate(ldr):
    
    cur_mean = torch.mean(x_tr)
    cur_std = torch.std(x_tr)
    mean.append(cur_mean)
    std.append(cur_std)
    print(cur_mean, cur_std)
print(np.mean(mean), np.mean(std))
